In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline

from ipywidgets import *
from IPython.display import display

## Building/Cleaning Data to Create CSV/DataFrames

In [2]:
# df = pd.read_csv('data/Indonesia/Indonesia.csv')

In [3]:
# df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# # missing value for 2020
# # filling with 75th percentile

# df['Ind_prod_60kg_bags'].fillna(df['Ind_prod_60kg_bags'].quantile(.75), inplace=True)

In [5]:
# df['ind_prod_lbs'] = [(each*1000)*2.2 for each in df['Ind_prod_60kg_bags']]

In [6]:
# df.drop('Ind_prod_60kg_bags', axis=1, inplace=True)

In [7]:
# df.to_csv('data/Indonesia/Indonesia2.csv')

In [8]:
# avg_temp = pd.read_csv('data/Indonesia/tas_timeseries_annual_cru_1901-2021_IDN.csv')

In [9]:
# avg_temp.drop(avg_temp.columns[[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34]], axis=1, inplace=True)

In [10]:
# avg_temp.drop(avg_temp.columns[[2]], axis=1, inplace=True)

In [11]:
# avg_temp2 = avg_temp.drop(avg_temp.index[0:89])

In [12]:
# avg_temp3 = avg_temp2.drop(avg_temp2.index[31])

In [13]:
# avg_temp3.rename(columns={'Variable:': 'Year'}, inplace=True)

In [14]:
# avg_temp3.rename(columns={'Indonesia': 'ind_avg_temp_celsius'}, inplace=True)

In [15]:
# ind = pd.merge(df, avg_temp3, on='Year')

In [16]:
# ind = ind[['Year', 'Temp Change', 'ind_avg_temp_celsius', 'ind_prod_lbs']]

In [17]:
# ind.rename(columns={'Year': 'year', 'Temp Change': 'temp_change'}, inplace=True)

In [18]:
# ind.to_csv('data/Indonesia/ind_final.csv')

In [19]:
# df = pd.read_csv('data/Indonesia/ind_final.csv')

In [20]:
# df.drop(df.columns[[0]], axis=1, inplace=True)

In [21]:
# df2 = pd.read_csv('data/cleaned_merged_df.csv')

In [22]:
# df.rename(columns={'temp_change': 'ind_annual_celcsius_change'}, inplace=True)

In [23]:
# cleaned_merged_df2 = pd.merge(df, df2, on='year')

In [24]:
# cleaned_merged_df2

In [25]:
# cleaned_merged_df2.to_csv('data/cleaned_merged_df2.csv')

In [26]:
# df2 = pd.read_csv('data/cleaned_merged_df3.csv')

In [27]:
# b_prod = pd.read_csv('data/production.csv')

In [28]:
# b_prod.drop(b_prod.columns[[0]], axis=1, inplace=True)

In [29]:
# b_prod.drop(b_prod.index[0:2], inplace=True)

In [30]:
# b_prod.drop(b_prod.index[1:56], inplace=True)

In [31]:
# df = b_prod.transpose()

In [32]:
# new_header = df.iloc[0]
# df = df[1:]
# df.columns = new_header

In [33]:
# df = df.reset_index()

In [34]:
# df.rename(columns={'index': 'year', 'Brazil': 'b_prod_60kg_bags'}, inplace=True)

In [35]:
# df.dtypes

In [36]:
# df['year'] = df['year'].astype(int)

In [37]:
# df['b_prod_lbs'] = [(each*1000)*2.2 for each in df['b_prod_60kg_bags']]

In [38]:
# df.drop(df.columns[[1]], axis=1, inplace=True)

In [39]:
# df['year'] = df['year'].str[:-3]

In [40]:
# cleaned_merged_df4 = pd.merge(df, df2, on='year')

In [41]:
# cleaned_merged_df4.drop(cleaned_merged_df4.columns[[2, 3]], axis=1, inplace=True)

In [42]:
# cleaned_merged_df4.head()

In [43]:
# cleaned_merged_df4.to_csv('data/cleaned_merged_df4.csv')

# Modeling

### Loading Data

In [44]:
df = pd.read_csv('data/cleaned_merged_df4.csv')

In [45]:
df.drop(df.columns[[0]], axis=1, inplace=True)

In [46]:
df.tail(3)

,year,b_prod_lbs,ind_annual_celcsius_change,ind_avg_temp_celsius,ind_prod_lbs,b_hectares_harvested,b_hectograms_per_hectare_yield,b_tonnes_produced,usa_retail_price,b_annual_avg_temp,...,col_annual_celsius_change,global_change_in_celsius,e_prod,e_temp_change,e_export,e_consumption,us_consumption_lbs,col_avg_temp_celsius,b_annual_celsius_change,e_avg_temp
27,2017,1.160276e+08,0.968,25.59,23875093.66,1802417,14894,2684508,4.450000,25.77,...,0.625,1.40,7346.7332,1.464,3773.406333,3643.0,57602600.0,25.11,1.348,23.65
28,2018,1.432878e+08,0.881,25.58,21158707.24,1863966,19060,3552729,4.300000,25.59,...,0.972,1.20,7511.2234,0.944,3589.048020,3685.0,58331521.6,25.04,1.170,23.27
29,2019,1.280636e+08,1.056,25.96,25152473.50,1825283,16500,3011745,4.141333,25.94,...,1.228,1.41,7343.4329,1.362,3921.249773,3781.0,59104449.8,25.42,1.532,23.41


### Train/Test Split for Temp (all 4 countries)

In [77]:
temp_features = [
    'ind_avg_temp_celsius', 
    'e_avg_temp', 
    'col_avg_temp_celsius',
    'b_annual_avg_temp'
]

In [78]:
df[temp_features].isnull().sum()

ind_avg_temp_celsius    0
e_avg_temp              0
col_avg_temp_celsius    0
b_annual_avg_temp       0
dtype: int64

In [79]:
X = df[temp_features]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Scaling

In [80]:
ss = StandardScaler()
print(ss.fit(X_train))

StandardScaler()


In [81]:
ss = StandardScaler()
print(ss.fit(X_test))

StandardScaler()


### Linear Regression for Temp (all 4 counties)

In [82]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.3734732342375028 -0.40131170280052086
MLR intercept: -21.112368586285967
MLR coeffs: [-1.88830049 -0.02702171 -0.20196791  3.09609429]


### Lasso for Temp (all 4 countries)

In [83]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.0 -0.4836121805558744
Lasso intercept: 3.622821300940439
Lasso coeffs: [0. 0. 0. 0.]


### Ridge for Temp (all 4 countries)

In [85]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.2740455820885115 -0.00419153000408401
Ridge intercept: -26.580509811165854
Ridge coeffs: [-0.02938096  0.39727081  0.16709224  0.69004647]


### Train/Test Split for Production (all 4 countries)

In [86]:
prod_features = [
    'ind_prod_lbs', 
    'col_production_lbs', 
    'e_prod',
    'b_prod_lbs'
]

In [87]:
df[prod_features].isnull().sum()

ind_prod_lbs          0
col_production_lbs    0
e_prod                0
b_prod_lbs            0
dtype: int64

In [88]:
X = df[prod_features]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Linear Regression for Production (all 4 countries)

In [89]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.44353998819682683 0.45966424192158284
MLR intercept: 2.843835978776019
MLR coeffs: [ 6.72053432e-08 -3.46340339e-08  1.08976633e-04 -3.16729409e-10]


#### Lasso for Production (all 4 countires)

In [90]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.4435372753882937 0.4601123754468267
Lasso intercept: 2.8448404561844898
Lasso coeffs: [ 6.74330637e-08 -3.47007468e-08  1.07135712e-04 -2.59935580e-10]


#### Ridge for Production (all 4 countries)

In [91]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.4435399881968267 0.4596642441740758
Ridge intercept: 2.843835983755618
Ridge coeffs: [ 6.72053444e-08 -3.46340342e-08  1.08976624e-04 -3.16729128e-10]


### Linear Regression for Individual Country Production

### Brazil

In [95]:
X = df[['b_prod_lbs']]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Linear Regression for Brazil Production

In [96]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.28947823626615643 0.3441955439413189
MLR intercept: 2.194464598065048
MLR coeffs: [1.64304545e-08]


#### Lasso for Brazil Production

In [97]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.28947823626615465 0.3441955084919954
Lasso intercept: 2.1944647175102343
Lasso coeffs: [1.64304531e-08]


#### Ridge for Brazil Production

In [98]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.28947823626615654 0.3441955439413188
Ridge intercept: 2.194464598065049
Ridge coeffs: [1.64304545e-08]


### Ethiopia

In [99]:
X = df[['e_prod']]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Linear Regression for Ethiopia Production

In [100]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.360119810998626 0.24580471817671146
MLR intercept: 2.2251372743615008
MLR coeffs: [0.00030887]


#### Lasso for Ethiopia Production

In [83]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.0 -0.4836121805558744
Lasso intercept: 3.622821300940439
Lasso coeffs: [0. 0. 0. 0.]


#### Ridge for Ethiopia Production

In [101]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.3601198109986258 0.2458047161471879
Ridge intercept: 2.2251372991582468
Ridge coeffs: [0.00030887]


C:\Users\micha\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- e_prod
Feature names seen at fit time, yet now missing:
- b_prod_lbs

  warnings.warn(message, FutureWarning)


### Colombia

In [102]:
X = df[['col_production_lbs']]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Linear Regression for Colombia Production

In [103]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.10123686974918555 -0.7878982106086718
MLR intercept: 4.88667404892861
MLR coeffs: [-4.82415064e-08]


#### Lasso for Colombia Production

In [105]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.1012368697491357 -0.7878979670005095
Lasso intercept: 4.886673161615404
Lasso coeffs: [-4.82414725e-08]


#### Ridge for Colombia Production

In [106]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.10123686974918555 -0.7878982106086705
Ridge intercept: 4.886674048928608
Ridge coeffs: [-4.82415064e-08]


### Indonesia

In [108]:
X = df[['ind_prod_lbs']]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Linear Regression for Indonesia Production

In [109]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.3538131649298706 0.5340857911865375
MLR intercept: 1.890872405104997
MLR coeffs: [9.53485117e-08]


#### Lasso for Indoesia Production

In [110]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Lasso score:', lasso.score(X_train, y_train), lasso.score(X_test, y_test))
print('Lasso intercept:', lasso.intercept_)
print('Lasso coeffs:', lasso.coef_)

Lasso score: 0.35381316492981474 0.534085553947214
Lasso intercept: 1.8908730927642938
Lasso coeffs: [9.53484739e-08]


#### Ridge for Indonesia Production

In [111]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

print('Ridge score:', ridge.score(X_train, y_train), ridge.score(X_test, y_test))
print('Ridge intercept:', ridge.intercept_)
print('Ridge coeffs:', ridge.coef_)

Ridge score: 0.35381316492987025 0.5340857911865364
Ridge intercept: 1.8908724051049999
Ridge coeffs: [9.53485117e-08]


## Gridsearch

In [65]:
# X = df.drop(['usa_retail_price'], axis=1)
# y = df['usa_retail_price']

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [66]:
# ss = StandardScaler()
# print(ss.fit(X_train))

In [67]:
# ss = StandardScaler()
# print(ss.fit(X_test))

In [68]:
# mlr = LinearRegression()
# mlr.fit(X_train, y_train)

# y_pred = mlr.predict(X_test)

# print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
# print('MLR intercept:', mlr.intercept_)
# print('MLR coeffs:', mlr.coef_)

In [69]:
# # Instantiate new pipeline object
# pipe = Pipeline([
#     ('pf', PolynomialFeatures(include_bias=False)),
#     ('ss', StandardScaler()),
#     ('mlr', LinearRegression())
# ])

# # Define dictionary of hyperparameters
# pipe_params = {
#     'ss__with_mean': [True, False],
#     'lasso__max_iter': [5000, 7500],
#     'lasso__alpha': [0.01, 0.1, 0.5, 1.0],
#     'lasso__selection': ['cyclic', 'random']
# }

# # Instantiate our GridSearchCV object
# gs = GridSearchCV(pipe, param_grid=pipe_params, verbose=1)

In [70]:
# pipe.get_params()